<a href="https://colab.research.google.com/github/salilathalye/chats-with-austin/blob/main/CWA_EDA_Sharpen_the_Saw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chats with Austin
###EDA - Sharpen the Saw
<p>
Time to put Pandas back into the rotation for a short EDA rust-removal, cleaning and sharpening. Basic stuff: DataFrame examination, groupby, crosstabs.
One of the things I still miss about the tidyverse is dplyr chaining using the maggritr pipe operator %>%. After several years (2013 - 2018) of building analytics and data science pipelines in R, I had developed a muscle-memory and state of "flow" in the tidyverse. I could envision what I needed to accomplish and the code would (almost) write itself. I still find method chaining a bit awkward in Pandas and everything seems a tad bit verbose...

Data is a modified version of the Titanic dataset from [DPhi](https://dphi.tech//). The questions I am researching come from a quiz in one of DPhi's learning modules.
<p>
Salil Athalye

www.bokehboy.com

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_url = 'https://raw.githubusercontent.com/dphi-official/First_ML_Model/master/titanic.csv'

In [3]:
dataset = pd.read_csv(data_url)

In [4]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


The Fare feature has 0 missing values


In [7]:
dataset[['Fare']].isna().sum()

Fare    0
dtype: int64

Are there more male passengers than female?


In [8]:
dataset[['Sex','PassengerId']].groupby('Sex')['PassengerId'].count()

Sex
female    314
male      577
Name: PassengerId, dtype: int64

In [ ]:
# Visual method

2. What is the proportion of passengers who survived?

In [9]:
total_passengers = dataset['PassengerId'].nunique()
total_survivors = dataset[dataset['Survived'] == 1]['PassengerId'].nunique()
proportion_survived = total_survivors / total_passengers
proportion_survived

0.3838383838383838

3. What is the median Fare of the passengers? 

In [10]:
dataset['Fare'].median()

14.4542

Proportion of survivors, men vs women


In [11]:
subset_by_sex = dataset[['PassengerId','Sex','Survived']].copy()
# axis=1 will run the proportions by row, so we get a proportion of survivors by sex
contingency_table = pd.crosstab(subset_by_sex['Sex'], subset_by_sex['Survived']).apply(lambda r: r/r.sum(), axis=1)
contingency_table

Survived,0,1
Sex,,
female,0.257962,0.742038
male,0.811092,0.188908


74% of women survived, compared to 19% of men

In [12]:
subset_by_class = dataset[['PassengerId','Pclass','Survived']].copy()
contingency_table = pd.crosstab(subset_by_class['Pclass'], subset_by_class['Survived']).apply(lambda r: r/r.sum(), axis=1)
contingency_table

Survived,0,1
Pclass,,
1,0.370370,0.629630
2,0.527174,0.472826
3,0.757637,0.242363


63% of First Class passengers survived, compared to 47% of Second Class and only 24% of Third Class passengers.


Proportion of Survivors by Age

In [13]:
subset_by_age = dataset[['PassengerId','Age','Survived']].copy()
bins= [0,20,120]
labels = ['Child','Adult']
subset_by_age['AgeGroup'] = pd.cut(subset_by_age['Age'], bins=bins, labels=labels, right=False)
contingency_table = pd.crosstab(subset_by_age['AgeGroup'], subset_by_class['Survived']).apply(lambda r: r/r.sum(), axis=1)
contingency_table

Survived,0,1
AgeGroup,,
Child,0.518293,0.481707
Adult,0.616364,0.383636


48% of Children survived compared to 38% of Adults.

How many of the survivors embarked from Southamption 'S'

In [14]:
survivors = dataset[dataset['Survived'] == 1]
survivors[survivors['Embarked'] == 'S'].shape[0]

217

6. Five highest fares of passengers (not unique)

In [15]:
subset = dataset[['Fare']].copy()
subset.sort_values(by='Fare', ascending=False).head(5)

,Fare
258,512.3292
737,512.3292
679,512.3292
88,263.0000
27,263.0000


Median Age of Passengers

In [16]:
dataset[['Age']].median()

Age    28.0
dtype: float64

Unique Names

In [17]:
dataset[['Name']].nunique()

Name    891
dtype: int64

Most of the passengers have x siblings/spouses

In [18]:
dataset[['SibSp']].mode()

,SibSp
0,0
